In [361]:
import pandas as pd

collisions = pd.read_csv('SDOT_Collisions_-_All_Years.csv', low_memory=False)
intersections = pd.read_csv('Intersections.csv')
traffic_signals = pd.read_csv('Traffic_Signals.csv')
poles = pd.read_csv('Seattle_City_Light_Poles.csv')

In [362]:
intersections = intersections.drop(['GIS_XCOORD',
                                    'OBJECTID',
                                    'GIS_YCOORD',
                                    'COMPKEY',
                                    'SUBAREA',
                                    'UNITID',
                                    'SHAPE_LNG',
                                    'SHAPE_LAT'
                                   ], axis = 'columns')

In [364]:
poles = poles.drop(['OBJECTID',
                    'ASSET_ID',
                    'SCL_SOURCE',
                    'PULLDATE',
                    'DDLAT',
                    'DDLON',
                    'FACILITY_NAME'
                   ], axis = 'columns')

In [365]:
traffic_signals = traffic_signals.drop(['OBJECTID',
                                        'COMPKEY',
                                        'COMPTYPE',
                                        'UNITID',
                                        'SEGKEY',
                                        'INTKEY',
                                        'INT_UNITID',
                                        'MAINT_AGREE',
                                        'PRIMARYDISTRICTCD',
                                        'SECONDARYDISTRICTCD'
                                       ], axis = 'columns')

In [366]:
#force to float
collisions = collisions.astype({"X": float, "Y": float})
intersections = intersections.astype({"X": float, "Y": float})
poles = poles.astype({"X": float, "Y": float})

# round columns X and Y to 5 decimal places
collisions['X'] = collisions['X'].apply(lambda x: round(x, 5))
collisions['Y'] = collisions['Y'].apply(lambda x: round(x, 5))

# round columns X and Y to 5 decimal places
intersections['X'] = intersections['X'].apply(lambda x: round(x, 5))
intersections['Y'] = intersections['Y'].apply(lambda x: round(x, 5))

# round columns X and Y to 5 decimal places
poles['X'] = poles['X'].apply(lambda x: round(x, 5))
poles['Y'] = poles['Y'].apply(lambda x: round(x, 5))

#removes trailing white space
traffic_signals['UNITDESC'] = traffic_signals['UNITDESC'].str.rstrip()

In [367]:
#filter collisions to only those within an intersection
collisions = collisions[collisions['ADDRTYPE'] == 'Intersection']

#provides number of collisions at X Y coordinates
collisions = collisions.groupby(['X', 'Y']).size().reset_index(name='collision_count')

In [368]:
#joining logic
collisions_intersections = pd.merge(collisions, intersections, how='left', on=['X', 'Y'])
collisions_intersections_signals = pd.merge(collisions_intersections, traffic_signals, how='left', on=['UNITDESC'])
collisions_intersections_signals_poles = pd.merge(collisions_intersections_signals, poles, how='left', on=['X', 'Y'])